In [1]:
# packages
import pandas as pd
import numpy as np

from claspy.segmentation import BinaryClaSPSegmentation

import ruptures as rpt

import stumpy
from aeon.segmentation import find_dominant_window_sizes

from aeon.segmentation import GreedyGaussianSegmenter

from aeon.segmentation import InformationGainSegmenter

from aeon.anomaly_detection import STRAY

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, make_scorer,mean_squared_error
from ruptures.metrics import precision_recall
import matplotlib.pyplot as plt
#from aeon.visualisation import plot_series_with_change_points, plot_series_with_profiles
import seaborn as sns

sns.set_theme()
sns.set_color_codes()

from claspy.tests.evaluation import f_measure,covering

from claspy.window_size import dominant_fourier_frequency, highest_autocorrelation, suss

C:\Users\loluc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\aeon\base\__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


In [2]:
#questi sono i dataset comprendente le timeseries e il groundtruth
timeseries=[#"in\cora1_input.txt",
      #"in\cora4_input.txt",
    "in\cora1_input.txt",
      "in\cora4_input.txt",
      "in\cora5_input.txt",
      "in\cora14_input.txt",
      "in\cora14_input.txt",
      "in\marianne7_input.txt",
      "in\marianne8_input.txt",
      "in\marianne10_input.txt",
      "in\marianne18_input.txt",
      "in\marianne19_input.txt",
      "in\marianne24_input.txt",
      "in\marianne26_input.txt",
      "in\marianne41_input.txt",
      "in\marianne42_input.txt",
      "in\marianne43_input.txt",
      "in\marianne47_input.txt",
      "in\marianne48_input.txt",
      "in\marianne48_input.txt",
      "in\muriel18_input.txt",
      "in\muriel26_input.txt",
      "in\muriel27_input.txt",
      "in\muriel30_input.txt"

      ]
groundtruth=[#"gt\cora_gt_2019-08-08_t001_video01.txt",
         #"gt\cora_gt_2019-08-08_t004_video01.txt",
            "gt\cora_gt_2019-08-08_t001_video01.txt",
         "gt\cora_gt_2019-08-08_t004_video01.txt",
         "gt\cora5_gt.txt",
         "gt\cora_gt_2019-08-08_t014_video01.txt",
         "gt\marianne_gt_2016-03-22_t007_video01.txt",
         "gt\marianne_gt_2016-03-22_t008_video01.txt",
         "gt\marianne_gt_2016-03-22_t010_video01.txt",
         "gt\marianne_gt_2016-03-22_t018_video01.txt",
         "gt\marianne_gt_2016-03-22_t019_video01.txt",
         "gt\marianne_gt_2016-03-22_t024_video01.txt",
         "gt\marianne_gt_2016-03-22_t026_video01.txt",
         "gt\marianne_gt_2016-03-22_t041_video01.txt",
         "gt\marianne_gt_2016-03-22_t042_video01.txt",
         "gt\marianne_gt_2016-03-22_t043_video01.txt",
         "gt\marianne_gt_2016-03-22_t047_video01.txt",
         "gt\marianne_gt_2016-03-22_t048_video01.txt",
         "gt\marianne_gt_2016-03-22_t048_video01.txt",
         "gt\muriel_gt_2016-03-21_t014_video01.txt",
         "gt\muriel_gt_2016-03-21_t018_video01.txt",
         "gt\muriel_gt_2016-03-21_t026_video01.txt",
         "gt\muriel_gt_2016-03-21_t027_video01.txt",
         "gt\muriel_gt_2016-03-23_t029_video01.txt",
         "gt\muriel_gt_2016-03-23_t030_video01.txt",
         "gt\muriel_gt_2016-03-23_t030_video01.txt"
         ]



In [9]:
df.iloc[:,0]

0        0.000000
1        0.014826
2        0.027840
3        0.040052
4        0.053237
          ...    
2933    44.061372
2934    44.076456
2935    44.090117
2936    44.104022
2937    44.117557
Name: 0, Length: 2938, dtype: float64

In [8]:
df.head()


,0,1,2,3,4,5,6,7,8,9,10
0,0.000000,0.000000,0.000000,0.0,584.989428,0.000000,0.000000,0.000000,0.000000,0.000000,0.239834
1,0.014826,0.005171,0.000000,0.0,584.861244,0.000000,0.000000,0.000000,0.000000,0.000000,0.239834
2,0.027840,0.496059,0.000001,0.0,585.052377,0.000147,0.000027,0.000001,0.000000,0.000031,0.128283
3,0.040052,0.138399,0.000003,0.0,585.256922,0.000737,0.000250,0.000000,0.000000,0.000434,0.128283
4,0.053237,0.187799,0.000004,0.0,585.275920,0.000581,0.000099,0.000079,0.000459,0.000140,0.128520


In [11]:
for i in range(len(timeseries)):
    df=pd.read_csv(timeseries[i],sep=' ', header=None)
    df.drop(0, axis=1)
    
    for j in [2,4,5,6,7,8,9,10]:
        ts=df.iloc[:,j]
        fft=dominant_fourier_frequency(ts.values)
        acf=highest_autocorrelation(ts.values)
        sus=suss(ts.values)

        f = open("outputFile/windowsizetest", "a")
        f.write(timeseries[i]+"\n")
        f.write("column:"+str(j)+"\n")
        f.write("fft:"+str(fft)+" acf:"+str(acf)+" suss:"+str(sus)+"\n")
        f.write("\n")
        f.close()

C:\Users\loluc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\claspy\window_size.py:213: UserWarning: Could not find any autocorrelation peaks. Using window_size=10.
  warnings.warn(f"Could not find any autocorrelation peaks. Using window_size={lbound}.")
